# 01. Heat Equation

Este ejemplo prueba la librería de resolución de EDPs con la ecuación del calor
$$
\left\{
\begin{array}{l}
\displaystyle \frac{\partial u}{\partial t} = k \partial \frac{\partial^2 u}{\partial x^2} \\
u(x,0)=f(x) \quad u(0,t)=0 \quad u(L,t) = 0
\end{array}
\right.
$$
donde tomaremos $f(x)= 6 \sin \left(\frac{\pi x}{L} \right)$, $L=2$, $k=1$. Además tomaremos $x \in [0,L], t \in [0,1]$.

La solución de la EPD es
$$
u(x,t)=6 \sin \left(\frac{\pi x}{L} \right) e^{-k \left(\frac{\pi}{L}\right)^2 t}
$$

In [7]:
# Iniciamos la clase comparador de MLP vs KAN
from edp_lib.edp_comparator import EDPComparator
import torch
import numpy as np

In [8]:
# Describimos la EDP usando la liberría
L=2
k=1

comparator = EDPComparator("2D") # Bajo este nombre se guardan los resultados
comparator.configure_variables({ # Configuramos el rango de las variables en el entrenamiento
    "x": (0, L),
    "t": (0, 1)
})
comparator.configure_outputs(["u"]) # Nombramos las funciones que debe aprender el modelo
comparator.configure_pde([lambda u_t, u_xx : u_t - k*u_xx]) # Definimos la edp
comparator.configure_boundaries([ # Definimos las condiciones de frontera
    ({ "x": 0}, lambda t: 0, "u"),
    ({ "x": L}, lambda t: 0, "u"),
    ({ "t": 0}, lambda x: 6*torch.sin((torch.pi*x)/L), "u")
])

# Configuramos la Kan
comparator.configure_kan_settings(grid=10, kan_width=[2,10,1])
comparator.configure_train_settings("kan", steps=100)

# Configuramos el Mlp
comparator.configure_mlp_settings(input_dim=2, output_dim=1, num_hidden_layers=3, hidden_dim=100)
comparator.configure_train_settings("mlp", steps=100)

# Definimos la solución real
comparator.set_real_solution({
    "u": lambda x,t: 6 * np.sin((np.pi * x) / L) * np.exp(-k * (np.pi / L)**2 * t)
})

In [9]:
# Ejecutamos la comparación
comparator.compare("comparation")

description: 100%|████████████████████████████████████████████████| 100/100 [00:42<00:00,  2.35it/s]


Image save as comparation/graphs/2D/u/kan.pdf
Image save as comparation/graphs/2D/u/mlp.pdf
Image save as comparation/graphs/2D/u/real.pdf
Image save as comparation/graphs/2D/u/comparation.pdf


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

In [10]:
# Obtenemos la diferencia numérica
comparator.get_results()

Total difference between kan model and real solution is 0.0000003632 for function u.
Total difference between mlp model and real solution is 0.0000007867 for function u.
